## A notbook for generating "It is not the case that... " data

In [1]:
import pandas as pd
import string

In [54]:
data_path = ''

In [45]:
df = pd.read_csv(data_path, '\t', error_bad_lines=False)#.dropna()


In [46]:
df = df.drop(columns=['label1', 'label2', 'label3', 'label4', 'label5']).dropna()

In [47]:
def it_is_not_the_case(sen):
    return "It is not the case that " + sen[0].lower() + sen[1:]

In [52]:
def get_implied_dfs(df):
    dff = df.copy()
    dff['neg_1'] = dff['sentence1'].transform(it_is_not_the_case)
    dff['neg_2'] = dff['sentence2'].transform(it_is_not_the_case)
    
    dfs = {}
    
    t1n1 = dff[['sentence2', 'neg_2']].copy().rename(columns={'sentence2': 'sentence1', 'neg_2': 'sentence2'})
    t1n1['gold_label'] = 'contradiction'
    dfs['t1n1c'] = t1n1
    
    en21 = dff[dff['gold_label']=='entailment'][['neg_2', 'sentence1']].copy().rename(columns={'neg_2': 'sentence1', 'sentence1': 'sentence2'})
    en21['gold_label'] = 'contradiction'
    dfs['en21c'] = en21

    en2n1 = dff[dff['gold_label']=='entailment'][['neg_2', 'neg_1']].copy().rename(columns={'neg_2': 'sentence1', 'neg_1': 'sentence2'})
    en2n1['gold_label'] = 'entailment'
    dfs['en2n1e'] = en2n1

    c2n1 = dff[dff['gold_label']=='contradiction'][['sentence2', 'neg_1']].copy().rename(columns={'sentence2': 'sentence1', 'neg_1': 'sentence2'})
    c2n1['gold_label'] = 'entailment'
    dfs['c2n1e'] = c2n1

    c1n2 = dff[dff['gold_label']=='contradiction'][['sentence1', 'neg_2']].copy().rename(columns={'sentence1': 'sentence1', 'neg_2': 'sentence2'})
    c1n2['gold_label'] = 'entailment'
    dfs['c1n2e'] = c1n2
    
    return dfs

In [53]:
dfs = get_implied_dfs(df)

for name in dfs:
    csv = open('snli_test_'+name+'.csv', 'w')
    dfs[name].to_csv(csv)